In [1]:
%reset

In [2]:
import polars as pl
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_tbl_rows(-1)

df_problems = pl.read_parquet('sampled_codeforces_data.parquet')
# df.write_parquet('codeforces_data.parquet')

In [3]:
df_problems.head(10)

shape: (10, 6)
┌─────────────────┬─────────────────┬─────────────────┬────────────────┬──────────┬────────────────┐
│ name            ┆ description     ┆ public_tests    ┆ private_tests  ┆ cf_index ┆ cf_tags        │
│ ---             ┆ ---             ┆ ---             ┆ ---            ┆ ---      ┆ ---            │
│ str             ┆ str             ┆ struct[2]       ┆ struct[2]      ┆ str      ┆ list[str]      │
╞═════════════════╪═════════════════╪═════════════════╪════════════════╪══════════╪════════════════╡
│ 500_C. New Year ┆ New Year is     ┆ {["3 5          ┆ {["50 50       ┆ C        ┆ ["constructive │
│ Book Reading    ┆ coming, and     ┆ 1 2 3           ┆ 75 71 23 37 28 ┆          ┆ algorithms",   │
│                 ┆ Jaehyun decided ┆ 1 3 2 3 1       ┆ 23 69 75 5 62  ┆          ┆ "greedy", …    │
│                 ┆ to read many    ┆ "],["12         ┆ 3 11 96 100 13 ┆          ┆ "math"]        │
│                 ┆ books during    ┆ "]}             ┆ 50 57 51 8 90  ┆          ┆                │
│                 ┆ 2015, unlike    ┆                 ┆ 4 6 84 27 11   ┆          ┆                │
│                 ┆ this year. He   ┆                 ┆ 89 95 81 10 62 ┆          ┆                │
│                 ┆ has n books     ┆                 ┆ 48 52 69 87 97 ┆          ┆                │
│                 ┆ numbered by     ┆                 ┆ 95 30 74 21 42 ┆          ┆                │
│                 ┆ integers from 1 ┆                 ┆ 36 64 31 80 81 ┆          ┆                │
│                 ┆ to n. The       ┆                 ┆ 50 56 53 33 99 ┆          ┆                │
│                 ┆ weight of the   ┆                 ┆ 26 30 5 33 35  ┆          ┆                │
│                 ┆ i-th (1 ≤ i ≤   ┆                 ┆ 29 6 15 36 17  ┆          ┆                │
│                 ┆ n) book is wi.  ┆                 ┆ 32 16 14 1 29  ┆          ┆                │
│                 ┆                 ┆                 ┆ 34 …           ┆          ┆                │
│                 ┆ As Jaehyun's …  ┆                 ┆                ┆          ┆                │
│ 643_B. Bear and ┆ Bearland has n  ┆ {["1000 999     ┆ {["1000 1001   ┆ B        ┆ ["constructive │
│ Two Paths       ┆ cities,         ┆ 10 20 30 40     ┆ 217 636 713    ┆          ┆ algorithms",   │
│                 ┆ numbered 1      ┆ ", "7 11        ┆ 516            ┆          ┆ "graphs"]      │
│                 ┆ through n.      ┆ 2 4 7 3         ┆ ", "1000 1998  ┆          ┆                │
│                 ┆ Cities are      ┆ "],["-1", "2 7  ┆ 833 681 19 233 ┆          ┆                │
│                 ┆ connected via   ┆ 1 5 6 3 4       ┆ ", … "4 5      ┆          ┆                │
│                 ┆ bidirectional   ┆ 7 2 1 5 6 4 3   ┆ 1 2 3 4        ┆          ┆                │
│                 ┆ roads. Each     ┆ "]}             ┆ "],["217 713 1 ┆          ┆                │
│                 ┆ road connects   ┆                 ┆ 2 3 4 5 6 7 8  ┆          ┆                │
│                 ┆ two distinct    ┆                 ┆ 9 10 11 12 13  ┆          ┆                │
│                 ┆ cities. No two  ┆                 ┆ 14 15 16 17 18 ┆          ┆                │
│                 ┆ roads connect   ┆                 ┆ 19 20 21 22 23 ┆          ┆                │
│                 ┆ the same pair   ┆                 ┆ 24 25 26 27 28 ┆          ┆                │
│                 ┆ of cities.      ┆                 ┆ 29 30 31 32 33 ┆          ┆                │
│                 ┆                 ┆                 ┆ 34 35 36 37 38 ┆          ┆                │
│                 ┆ Bear Limak was  ┆                 ┆ 39 40 …        ┆          ┆                │
│                 ┆ once in…        ┆                 ┆                ┆          ┆                │
│ 774_D. Lie or   ┆ Vasya has a     ┆ {["4 2 4        ┆ {["5 1 3       ┆ D        ┆ ["*special",   │
│ Truth           ┆ sequence of     ┆ 1 1 1 1 

In [4]:
df_problems["name"][114]

'1102_D. Balanced Ternary String'

In [5]:
# Create df for statistics: pass_k (k_tries + 1), k_tries (number of debugging processes), exec_time, total_llm_calls, total_input_tokens, total_output_tokens, total_tokens, total_costs_usd, number_given_pub_tests, algorithm_name

# Step 1: Create a new empty DataFrame with columns copied from the existing df

# schema = {
#     "name": pl.Utf8,
#     "description": pl.Utf8,
#     "public_tests": pl.Struct([pl.Field("input", pl.List(pl.Utf8)), pl.Field("output", pl.List(pl.Utf8))]),
#     "private_tests": pl.Struct([pl.Field("input", pl.List(pl.Utf8)), pl.Field("output", pl.List(pl.Utf8))]),
#     "cf_index": pl.Utf8,
#     "cf_tags": pl.List(pl.Utf8),
#     "solution_algorithm_name": pl.Utf8,
#     "pass_k": pl.Int64,
#     "k_tries": pl.Int64,
#     "exec_time_sec": pl.Float64,
#     "total_llm_calls": pl.Int64,
#     "total_input_tokens": pl.Int64,
#     "total_output_tokens": pl.Int64,
#     "total_tokens": pl.Int64,
#     "total_costs_usd": pl.Float64,
#     "k_debug": pl.Int64,
#     "t_plan": pl.Int64,
#     "programming_language": pl.Utf8,
#     "code_solution": pl.Utf8,
#     "is_solved": pl.Boolean,
#     "number_given_pub_tests": pl.Int64,
#     "total_number_test_cases": pl.Int64,
#     "number_failed_test_cases": pl.Int64,
# }
# 
# df_statistics = pl.DataFrame(schema=schema)

In [6]:
df_statistics = pl.read_parquet("/Users/maksim/Documents/VSE/Bachelor's thesis/self-corrective-coding-agent/data/experiment_results.parquet")

In [21]:
from langchain_core.runnables import RunnableConfig
import time
from langchain_community.callbacks import get_openai_callback
from agent.graph.utils.state import TestCase
from typing import Dict, List
from agent.graph.graph import graph
from dotenv import load_dotenv
from tqdm import tqdm


# Load API keys from .env
load_dotenv()

def convert_to_test_case(tests_dict: Dict, is_public: bool = False) -> List[TestCase]:
    return [
        TestCase(input=inp, expected_output=out, is_public=is_public)
        for inp, out in zip(tests_dict.get('input', []), tests_dict.get('output', []))
    ]

# Setup variables K, T
programming_language = 'python'
K = 3
T = 3
config = RunnableConfig(recursion_limit=100)

output_file_path = "/Users/maksim/Documents/VSE/Bachelor's thesis/self-corrective-coding-agent/data/experiment_results.parquet"
failed_indexes_path = "/Users/maksim/Documents/VSE/Bachelor's thesis/self-corrective-coding-agent/data/failed_indexes.csv"

failed_indexes = [40, 100]  # add "You're trying to set the record on your favorite video game.", 115 wakanda? 

In [22]:
# Start iteration over problems
async def process(i, df_problems):
    input_state = {
        "original_problem": df_problems['description'][i],
        "public_tests": convert_to_test_case(df_problems['public_tests'][i]),
        "private_tests": convert_to_test_case(df_problems['private_tests'][i]),
        "programming_language": programming_language,
        "k_debug": K,
        "t_plan": T,
    }
    
    # Track starting time
    start_time = time.time()
    try:
        with get_openai_callback() as cb:
            final_state = await graph.ainvoke(input_state, config=config)
    except Exception as e:
        raise e
    
    # Calculate execution time
    end_time = time.time()
    execution_time = round(end_time - start_time, 2)
    
    # Append callbacks statistics and time to the statistics dataframe
    plans_sorted = final_state.get("gen_plans").plans
    t = final_state.get("t_current") if final_state.get("t_current") < T else T-1
    solution_algorithm_name = plans_sorted[t].algorithm_name
    
    row_data = {
        "name": df_problems['name'][i],
        "description": df_problems['description'][i],
        "public_tests": df_problems['public_tests'][i],
        "private_tests": df_problems['private_tests'][i],
        "cf_index": df_problems['cf_index'][i],
        "cf_tags": df_problems['cf_tags'][i],
        "solution_algorithm_name": solution_algorithm_name,
        "pass_k": final_state.get("k_tries") + 1,
        "k_tries": final_state.get("k_tries"),
        "exec_time_sec": execution_time,
        "total_llm_calls": cb.successful_requests,
        "total_input_tokens": cb.prompt_tokens,
        "total_output_tokens": cb.completion_tokens,
        "total_tokens": cb.total_tokens,
        "total_costs_usd": cb.total_cost,
        "k_debug": K,
        "t_plan": T,
        "programming_language": programming_language,
        "code_solution": final_state.get('code'),
        "is_solved": final_state.get('test_evaluation_result').all_tests_passed,
        "number_given_pub_tests": len(df_problems['public_tests'][i]["input"]),
        "total_number_test_cases": len(final_state.get('public_tests')) + len(final_state.get('private_tests')),
        "number_failed_test_cases": len(final_state.get('test_evaluation_result').debug_info.failed_cases) if not final_state.get('test_evaluation_result').all_tests_passed else 0,
    }

    return row_data
    
    


In [23]:
import asyncio
from tqdm import tqdm
import polars as pl
import time
from typing import Dict, List, Any

class ProcessTimeoutError(Exception):
    """Custom exception for process timeout"""
    pass

async def process_with_timeout(i: int, df_problems: pl.DataFrame, timeout: int) -> Dict[str, Any]:
    try:
        async with asyncio.timeout(timeout):
            return await process(i, df_problems)
    except asyncio.TimeoutError:
        raise ProcessTimeoutError(f"Process timed out after {timeout} seconds")


In [24]:
# Select rows with missing names, retaining the original index
missing_names_with_index = df_problems.filter(~pl.col("name").is_in(df_statistics["name"]))
missing_names_with_index

name,description,public_tests,private_tests,cf_index,cf_tags
str,str,struct[2],struct[2],str,list[str]
"""396_E. On Iteration of One Well-Known Function""","""Of course, many of you can calculate φ(n) — the number of positive integers that are less than or equal to n, that are coprime with n. But what if we need to calculate φ(φ(...φ(n))), where function φ …","{[""1 7 1 2 "", ""1 2 100000000000000000 10000000000000000 "", ""1 7 1 1 ""],[""1 2 1 "", ""1 2 90000000000000000 "", ""2 2 1 3 1 ""]}","{[""14 748379 98840576926665825 813677 13311059299492 815341 1411005636 819773 68152087990533 842489 1075388131378226 843907 28789 853529 4806007884 858269 22414705800 859049 147 897577 1484807 942883 4236123 949931 2543157177406 971683 9 992183 19323 10 "", ""15 384533 874985842 533831 12289 583909 34216634719 617971 22529424 694541 53162325244602684 747781 1 764149 46835688011218563 843503 515749916879 880559 45 881357 13 922549 1430350625759 943543 5480946198 948799 1342122 972557 209220 991031 46976242 10 "", … ""20 617249 95505350773622504 669359 10 719009 116668174 763403 54968 765461 5853021 782141 3255937150271872 827269 33475224309 837779 13606358599462 871763 222 879881 819254389504729 922289 202935304 926171 6 941771 43605 964289 118766 964697 5087819909627 971483 400727314518461 983783 27553897 984167 773 985351 2362345616 988829 2774674 10000000000000 ""],[""52 2 740 3 242 5 98 7 78 11 15 13 10 19 10 23 26 29 11 37 10 41 1 43 9 47 10 53 10 67 1 71 1 83 10 107 17 127 1 139 10 149 1 167 11 191 1 251 1 269 1 283 11 331 1 491 1 643 1 673 1 941 1 1499 1 1979 1 1987 1 2999 1 3767 1 94993 1 203419 1 374189 1 748379 98840576926665815 813677 13311059299482 815341 1411005626 819773 68152087990523 842489 1075388131378216 843907 28779 853529 4806007874 858269 22414705790 859049 137 897577 1484797 942883 4236113 949931 2543157177396 992183 19313 "", ""50 2 700 3 229 5 153 7 111 11 78 13 34 17 2 19 9 23 11 29 10 41 9 43 10 59 10 61 1 79 1 83 1 101 1 131 1 173 1 191 1 197 1 199 1 211 1 241 1 251 10 317 1 383 1 827 1 1667 1 3433 1 3931 1 20599 1 31477 1 52711 1 62897 1 99103 1 384533 874985832 533831 12279 583909 34216634709 617971 22529414 694541 53162325244602674 764149 46835688011218553 843503 515749916869 880559 35 881357 3 922549 1430350625749 943543 5480946188 948799 1342112 972557 209210 991031 46976232 "", … ""24 2 566267904935543 3 60211741392905 5 60009830252538 7 10000208528260 11 10033475280048 13 15121301105727 17 20000119553796 23 5853021 47 9999999999999 241 33475224309 2381 1 2411 1 19289 1 19553 1 21997 1 28573 1 39107 1 52361 1 418889 1 617249 95495350773622504 782141 3245937150271872 837779 3606358599462 879881 809254389504729 971483 390727314518461 ""]}","""E""","[""math""]"
"""14_D. Two Paths""","""As you know, Bob's brother lives in Flatland. In Flatland there are n cities, connected by n - 1 two-way roads. The cities are numbered from 1 to n. You can get from one city to another moving along t…","{[""6 1 2 2 3 2 4 5 4 6 4 "", ""7 1 2 1 3 1 4 1 5 1 6 1 7 "", ""4 1 2 2 3 3 4 ""],[""4"", ""0"", ""1""]}","{[""4 3 2 3 4 1 4 "", ""6 1 2 2 5 4 5 4 6 3 2 "", … ""5 2 4 2 5 1 5 2 3 ""],[""1"", ""4"", … ""2""]}","""D""","[""dfs and similar"", ""dp"", … ""two pointers""]"
"""107_E. Darts""","""The night after the graduation ceremony graduate students of German University in Cairo (GUC) are playing darts. As there's no real dart board available, the photographs of members of the GUC upper ma…","{[""4 0 0 0 1 3 1 3 0 0 0 0 3 1 3 1 0 3 3 2 3 2 0 3 0 3 3 3 2 0 2 0 3 "", ""2 -1 0 0 1 1 0 0 -1 0 0 1 1 2 0 1 -1 "", … ""1 -1 0 0 1 1 0 0 -1 ""],[""1.5000000000 "", ""1.1428571429 "", … ""1.0000000000 ""]}","{[""2 0 0 1 0 1 1 0 1 10 0 11 0 11 1 10 1 "", ""2 0 0 1 0 1 1 0 1 0 10 1 10 1 11 0 11 "", … ""2 0 0 0 1 1 1 1 0 0 -1 0 0 1 0 1 -1 ""],[""1.0000000000 "", ""1.0000000000 "", … ""1.0000000000 ""]}","""E""","[""geometry"", ""probabilities""]"


In [25]:
df_problems = missing_names_with_index
df_problems

name,description,public_tests,private_tests,cf_index,cf_tags
str,str,struct[2],struct[2],str,list[str]
"""396_E. On Iteration of One Well-Known Function""","""Of course, many of you can calculate φ(n) — the number of positive integers that are less than or equal to n, that are coprime with n. But what if we need to calculate φ(φ(...φ(n))), where function φ …","{[""1 7 1 2 "", ""1 2 100000000000000000 10000000000000000 "", ""1 7 1 1 ""],[""1 2 1 "", ""1 2 90000000000000000 "", ""2 2 1 3 1 ""]}","{[""14 748379 98840576926665825 813677 13311059299492 815341 1411005636 819773 68152087990533 842489 1075388131378226 843907 28789 853529 4806007884 858269 22414705800 859049 147 897577 1484807 942883 4236123 949931 2543157177406 971683 9 992183 19323 10 "", ""15 384533 874985842 533831 12289 583909 34216634719 617971 22529424 694541 53162325244602684 747781 1 764149 46835688011218563 843503 515749916879 880559 45 881357 13 922549 1430350625759 943543 5480946198 948799 1342122 972557 209220 991031 46976242 10 "", … ""20 617249 95505350773622504 669359 10 719009 116668174 763403 54968 765461 5853021 782141 3255937150271872 827269 33475224309 837779 13606358599462 871763 222 879881 819254389504729 922289 202935304 926171 6 941771 43605 964289 118766 964697 5087819909627 971483 400727314518461 983783 27553897 984167 773 985351 2362345616 988829 2774674 10000000000000 ""],[""52 2 740 3 242 5 98 7 78 11 15 13 10 19 10 23 26 29 11 37 10 41 1 43 9 47 10 53 10 67 1 71 1 83 10 107 17 127 1 139 10 149 1 167 11 191 1 251 1 269 1 283 11 331 1 491 1 643 1 673 1 941 1 1499 1 1979 1 1987 1 2999 1 3767 1 94993 1 203419 1 374189 1 748379 98840576926665815 813677 13311059299482 815341 1411005626 819773 68152087990523 842489 1075388131378216 843907 28779 853529 4806007874 858269 22414705790 859049 137 897577 1484797 942883 4236113 949931 2543157177396 992183 19313 "", ""50 2 700 3 229 5 153 7 111 11 78 13 34 17 2 19 9 23 11 29 10 41 9 43 10 59 10 61 1 79 1 83 1 101 1 131 1 173 1 191 1 197 1 199 1 211 1 241 1 251 10 317 1 383 1 827 1 1667 1 3433 1 3931 1 20599 1 31477 1 52711 1 62897 1 99103 1 384533 874985832 533831 12279 583909 34216634709 617971 22529414 694541 53162325244602674 764149 46835688011218553 843503 515749916869 880559 35 881357 3 922549 1430350625749 943543 5480946188 948799 1342112 972557 209210 991031 46976232 "", … ""24 2 566267904935543 3 60211741392905 5 60009830252538 7 10000208528260 11 10033475280048 13 15121301105727 17 20000119553796 23 5853021 47 9999999999999 241 33475224309 2381 1 2411 1 19289 1 19553 1 21997 1 28573 1 39107 1 52361 1 418889 1 617249 95495350773622504 782141 3245937150271872 837779 3606358599462 879881 809254389504729 971483 390727314518461 ""]}","""E""","[""math""]"
"""14_D. Two Paths""","""As you know, Bob's brother lives in Flatland. In Flatland there are n cities, connected by n - 1 two-way roads. The cities are numbered from 1 to n. You can get from one city to another moving along t…","{[""6 1 2 2 3 2 4 5 4 6 4 "", ""7 1 2 1 3 1 4 1 5 1 6 1 7 "", ""4 1 2 2 3 3 4 ""],[""4"", ""0"", ""1""]}","{[""4 3 2 3 4 1 4 "", ""6 1 2 2 5 4 5 4 6 3 2 "", … ""5 2 4 2 5 1 5 2 3 ""],[""1"", ""4"", … ""2""]}","""D""","[""dfs and similar"", ""dp"", … ""two pointers""]"
"""107_E. Darts""","""The night after the graduation ceremony graduate students of German University in Cairo (GUC) are playing darts. As there's no real dart board available, the photographs of members of the GUC upper ma…","{[""4 0 0 0 1 3 1 3 0 0 0 0 3 1 3 1 0 3 3 2 3 2 0 3 0 3 3 3 2 0 2 0 3 "", ""2 -1 0 0 1 1 0 0 -1 0 0 1 1 2 0 1 -1 "", … ""1 -1 0 0 1 1 0 0 -1 ""],[""1.5000000000 "", ""1.1428571429 "", … ""1.0000000000 ""]}","{[""2 0 0 1 0 1 1 0 1 10 0 11 0 11 1 10 1 "", ""2 0 0 1 0 1 1 0 1 0 10 1 10 1 11 0 11 "", … ""2 0 0 0 1 1 1 1 0 0 -1 0 0 1 0 1 -1 ""],[""1.0000000000 "", ""1.0000000000 "", … ""1.0000000000 ""]}","""E""","[""geometry"", ""probabilities""]"


In [12]:
for i in tqdm(range(1, df_problems.height), desc="Processing Problems", unit="problem", ncols=80):
    try:
        row_data = await process_with_timeout(i, df_problems, timeout=600)
    except Exception as e:
        print(f"Error with problem index {i}: {e}")
        failed_indexes.append(i)
        continue

    row_df = pl.DataFrame([row_data])
    df_statistics = pl.concat([df_statistics, row_df], how="vertical")
    df_statistics.write_parquet(output_file_path)
    time.sleep(1)

failed_df = pl.DataFrame({"failed_problem_indexes": failed_indexes})
failed_df.write_csv(failed_indexes_path)

Processing Problems: 100%|██████████████████| 5/5 [17:59<00:00, 215.85s/problem]


In [13]:
df_statistics['name'].value_counts()


name,count
str,u32
"""1242_D. Number Discovery""",1
"""1078_B. The Unbearable Lightness of Weights""",1
"""1183_F. Topforces Strikes Back""",1
"""734_F. Anton and School""",1
"""282_A. Bit++""",1
"""402_B. Trees in a Row""",1
"""487_A. Fight the Monster""",1
"""756_F. Long number""",1
"""711_C. Coloring Trees""",1


In [14]:
"""
Stats and charts:
pass_k(1 and 3)/ num of solved overall, pass_k(1 and 3)/ num of solved per difficulty level, k tries for solved, number of solved and failed tests for failed, total and average llm calls for solved, average time to solve, average usd cost for solved, average token cost for solved, algorithm types histogram, algorithm types correctly matches one of in cf_tags column, average failed tests per dif level, check solve rate dependency on number of given public tests
"""

'\nStats and charts:\npass_k(1 and 3)/ num of solved overall, pass_k(1 and 3)/ num of solved per difficulty level, k tries for solved, number of solved and failed tests for failed, total and average llm calls for solved, average time to solve, average usd cost for solved, average token cost for solved, algorithm types histogram, algorithm types correctly matches one of in cf_tags column, average failed tests per dif level, check solve rate dependency on number of given public tests\n'